In [127]:
import nltk
from textblob import TextBlob as tb
import math
import io
import codecs
from __future__ import division
import numpy
import pandas as pd
import matplotlib
%matplotlib inline 


In [128]:
#http://stevenloria.com/finding-important-words-in-a-document-using-tf-idf/

#computes "term frequency" which is the number of times a word appears in a document blob, 
#normalized by dividing by the total number of words in blob. 
def tf1(word, blob):
    return blob.words.count(word)

def tf2(word, blob):
    return blob.words.count(word) / len(blob.words)

#returns the number of documents containing word. A generator expression is passed to the sum() function.
def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

#computes "inverse document frequency" which measures how common a word is among all documents in bloblist. 
#The more common a word is, the lower its idf. 
#We take the ratio of the total number of documents to the number of documents containing word, 
#then take the log of that. Add 1 to the divisor to prevent division by zero.
def idf(word, bloblist):
     return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

#computes the TF-IDF score. It is simply the product of  tf and idf.
def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [ ]:
import sys
import re
import os
import string
import re
os.chdir('/users/asheets/Documents/Work_Computer_new/Work_Computer/Grad_School/PREDICT_453/Notebooks/DSI/')

all_docs = []
blob_list = []
d = 1

cachedStopWords = nltk.corpus.stopwords.words('english')
pattern = re.compile(r'\b(' + r'|'.join(cachedStopWords) + r')\b\s*')

for i in range(40):
    doc_name = 'DSI' + str(d) + '.txt'
    try:
        with open(doc_name, 'r') as f:
            sample = f.read()
        sample = sample.decode('utf-8')
       #sample = sample.decode('ascii')
        sample = sample.lower()
        sample = re.sub(r'[^\w]', ' ', sample)
        sample = ''.join([i for i in sample if not i.isdigit()])
        sample = pattern.sub('', sample)
        sample = "".join(l for l in sample if l not in string.punctuation)
        sample2 = " ".join(k for k in tb(sample).noun_phrases)
        all_docs.append({'DSInum': d, 'raw_text': sample, 'noun_phrases_only':
        sample2})
        blob_list.append(tb(sample2))
        d = d + 1
    except IOError:
        d = d + 1
        pass


In [138]:
#compare my article
blob1 = tb(all_documents['noun_phrases_only'][23])
term_freq = [tf1(word,blob1) for word in blob1.words]
term_rel_freq = [tf2(word,blob1) for word in blob1.words]
tf_df1 = pd.DataFrame({'word': blob1.words, 'doc1_term_freq': term_freq, 'doc1_term_rel_freq': term_rel_freq})

docs_containing1 = pd.DataFrame({'word': blob1.words, 'doc_freq': [n_containing(word,blob_list) for word in blob1.words]})

df = pd.merge(tf_df1,docs_containing1,on='word',how='outer').drop_duplicates()

df['intermediate_calc'] = (len(blob_list) / df['doc_freq']).astype(float)
df['idf'] = df['intermediate_calc'].apply(math.log)
df = df.sort_values("idf",ascending=True)

tfidf_df = pd.DataFrame({'word': blob1.words, 'tf_idf_doc1': [tfidf(word, blob1, blob_list) for word in blob1.words]})

df = pd.merge(df,tfidf_df,on='word',how='outer').drop_duplicates()
df_final = df[["word","doc1_term_freq","doc1_term_rel_freq","doc_freq","idf","tf_idf_doc1"]]
df_final.sort_values(by=['tf_idf_doc1'], ascending=[False]).head(n=10)
df_final.to_csv("/users/asheets/Documents/Work_Computer_new/Work_Computer/Grad_School/PREDICT_453/Notebooks/DSI24_tfidf.txt", sep='\t')

In [140]:
##Compare just the two articles we know to be similar
blob2 = tb(all_documents['noun_phrases_only'][27])
term_freq = [tf1(word,blob2) for word in blob2.words]
term_rel_freq = [tf2(word,blob2) for word in blob2.words]
tf_df2 = pd.DataFrame({'word': blob2.words, 'doc2_term_freq': term_freq, 'doc2_term_rel_freq': term_rel_freq})
two_tf = pd.merge(tf_df1,tf_df2,on='word',how='outer').drop_duplicates()

docs_containing1 = pd.DataFrame({'word': blob1.words, 'doc_freq': [n_containing(word,bloblist) for word in blob1.words]})
docs_containing2 = pd.DataFrame({'word': blob2.words, 'doc_freq': [n_containing(word,bloblist) for word in blob2.words]})

doc_freq = pd.concat([docs_containing1,docs_containing2]).drop_duplicates().reset_index(drop=True)
df = pd.merge(doc_freq,two_tf,on='word',how='inner')
df = df.sort_values("doc_freq",ascending=False)

df['intermediate_calc'] = (len(blob_list) / df['doc_freq']).astype(float)
df['idf'] = df['intermediate_calc'].apply(math.log)
df = df.sort_values("idf",ascending=False)

scores1 = pd.DataFrame({'word': blob1.words, 'tf_idf_doc1': [tfidf(word, blob1, blob_list) for word in blob1.words]})
scores2 = pd.DataFrame({'word': blob2.words, 'tf_idf_doc2': [tfidf(word, blob2, blob_list) for word in blob2.words]})

two_tfidf= pd.merge(scores1,scores2,on='word',how='outer').drop_duplicates()
two_tfidf = two_tfidf[['word', 'tf_idf_doc1', 'tf_idf_doc2']]
two_tfidf = two_tfidf.sort_values(by=['tf_idf_doc1', 'tf_idf_doc2'], ascending=[False,False])
two_tfidf.head(n=6)


,word,tf_idf_doc1,tf_idf_doc2
192,energy,0.050831,0.039822
266,era,0.034877,0.011385
473,future,0.032492,NaN
329,coal,0.028441,0.027851
122,order,0.027793,0.036289
388,emissions,0.025994,0.025455


In [141]:
#compare all DSIs sing all pre-defined functions
DSI_list = all_documents["DSInum"]
for i, blob in enumerate(blob_list):
    print("Top words in document {}".format(DSI_list[i]))
    scores = {word: tfidf(word, blob, blob_list) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:10]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))
        
        

Top words in document 1
	Word: registry, TF-IDF: 0.06249
	Word: muslim, TF-IDF: 0.05373
	Word: countries, TF-IDF: 0.03303
	Word: immigration, TF-IDF: 0.03204
	Word: source, TF-IDF: 0.03201
	Word: risk, TF-IDF: 0.03201
	Word: ban, TF-IDF: 0.03201
	Word: terrorism, TF-IDF: 0.03125
	Word: muslims, TF-IDF: 0.03125
	Word: database, TF-IDF: 0.03125
Top words in document 2
	Word: nuclear, TF-IDF: 0.16595
	Word: arms, TF-IDF: 0.06915
	Word: race, TF-IDF: 0.04756
	Word: proliferation, TF-IDF: 0.04149
	Word: corp, TF-IDF: 0.04149
	Word: uranium, TF-IDF: 0.04149
	Word: treaty, TF-IDF: 0.04149
	Word: friday, TF-IDF: 0.0398
	Word: weapons, TF-IDF: 0.03567
	Word: comments, TF-IDF: 0.03429
Top words in document 3
	Word: act, TF-IDF: 0.06712
	Word: infrastructure, TF-IDF: 0.02937
	Word: ban, TF-IDF: 0.02427
	Word: corruption, TF-IDF: 0.02369
	Word: workers, TF-IDF: 0.02203
	Word: american, TF-IDF: 0.02153
	Word: clean, TF-IDF: 0.01849
	Word: education, TF-IDF: 0.01801
	Word: constitutional, TF-IDF: 0.

In [151]:
RTV = pd.read_csv('RTV.txt',header=None)
RTV.columns = ['word']

RTVblob = tb(str(tuple(RTV.word.tolist())).replace("'", ""))
RTV_final = list(RTV['word'])

tf_list = []

for i, blob in enumerate(blob_list):
    for item in RTV_final:
        tf_list.append({'DSInum': DSI_list[i], 'word': item, 'term_freq': blob.words.count(item)})

#print pd.DataFrame(tf_list)
tf_df = pd.DataFrame(tf_list)
tf_df.to_csv("RTV_frequencies.txt", sep='\t')

new_tf = tf_df[tf_df['DSInum'] == 24]
new_tf = new_tf[["word","term_freq"]]

for i in range(1,36):
    try:
        tmp = tf_df[tf_df['DSInum'] == DSI_list[i]]
        tmp = tmp[["word","term_freq"]]
        new_tf = pd.merge(new_tf, tmp, on='word', how='outer')
    except IOError:
        pass  

new_tf[new_tf == 0] = numpy.NaN
print new_tf
#new_tf.to_csv("RTV_frequencies2.txt", sep='\t')

ValueError: cannot reindex from a duplicate axis

0     24
1     25
2     26
3     27
4     28
5     29
6     30
7     31
8     32
9     33
10    34
11    36
12    39
Name: DSInum, dtype: int64
